# Downloading data from Google Drive

In [ ]:
#!pip install gdown


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi   ( saber_11_pro.csv)
#url = 'https://drive.google.com/uc?export=download&id=1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi'
#output = 'saber_11_pro.csv'
#gdown.download(url, output, quiet=False)


In [ ]:
#import gdown
# to download the data from Google Drive
# id = 10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu'
#output = 'saber_combined_all_fields.csv'
#gdown.download(url, output, quiet=False)

In [ ]:
#import gdown
# to download the data from Google Drive
# id = 1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV'
#output = 'saber_combined_preprocessed.csv'
#gdown.download(url, output, quiet=False)

In [2]:
!pip install shap
!pip install lime

# Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import plotly.express as px
import plotly.graph_objs as go

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score,  r2_score,  f1_score, classification_report,roc_curve,roc_auc_score


 
from sklearn import model_selection  # Kfold..
# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import warnings

pd.set_option('display.max_rows', 200) # to show more rows.
warnings.simplefilter('ignore')



# DF, Dataset : Saber_Combined_Preprocessed

In [4]:
df = pd.read_csv('../input/saber-combined-preprocessed/saber_combined_preprocessed.csv')
df.head()

In [ ]:
#df.info()

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [6]:
df = reduce_mem_usage(df)

In [ ]:
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data

nan_check(df)

In [7]:
replace_dict = {'AGRICULTURAL, FORESTRY ENGINEERING':'AGRICULTURAL, FOREST ENGINEERING',
               'AGROINDUSTRIAL ENGINEERING, FOOD':'AGROINDUSTRIAL AND FOOD ENGINEERING',
               'TRAINING RELATED TO THE MILITARY OR POLICE FIELD':'MILITARY OR POLICE TRAINING',
               'NUTRITION AND DIET':'NUTRITION AND DIETETICS'}
df['your_undergraduate_core'] = df['your_undergraduate_core'].replace(replace_dict)

In [8]:
df['your_undergraduate_core'].value_counts()

In [9]:
df = df[df['your_undergraduate_core'] != 'UNCLASSIFIED']
df = df[df['your_undergraduate_core'] != 'UNKNOWN']
df['your_undergraduate_core'].value_counts()


In [10]:
# dictionary of categorical variables
import urllib.request as urllib2
import ast
contents = str("")
for line in urllib2.urlopen("https://raw.githubusercontent.com/vcuspinera/Datasets/main/omdena/colombia-career-recommender-system/translate_cat.txt"):
    contents += str(line)[2:-3]
contents += "}"
char_to_replace = {
    "\\r": "", "\\": "", # characters
    "xc3x81": "Á", "xc3x89": "É", "xc3x8d": "Í", "xc3x93": "Ó", "xc3x9a": "Ú", "xc3x9c": "Ü", "xc3x91": "Ñ", # capital letters
    "xc3xa1": "á", "xc3xa9": "é", "xc3xad": "í", "xc3xb3": "ó", "xc3xba": "ú", "xc3xb1": "ñ" # lowercase letters
}
for key, value in char_to_replace.items():
    contents = contents.replace(key, value)
dic_cat = ast.literal_eval(contents)
# added these to ENGINEERING
dic_cat['your_undergraduate_core_field']['AGRICULTURAL, FOREST ENGINEERING']='ENGINEERING'
dic_cat['your_undergraduate_core_field']['AGRONOMIC, LIVESTOCK ENGINEERING']='ENGINEERING'
dic_cat['your_undergraduate_core_field']['PHISICS']='MATH AND PHYSICAL SCIENCES'
#dic_cat['your_undergraduate_core_field']

In [11]:
dic_cat['your_undergraduate_core_field']


In [12]:
# Add the undergraduate core field variable
df['your_undergraduate_core_field'] = df[['your_undergraduate_core']].replace(dic_cat['your_undergraduate_core_field'])
df.head()

In [13]:
df['your_undergraduate_core_field']  = df['your_undergraduate_core_field'].replace({'PLASTIC, VISUAL ARTS':'PLASTIC ARTS, VISUAL ARTS'})
df['your_undergraduate_core_field'].value_counts()

# Using Altair

In [14]:
# to plot
import altair as alt
alt.renderers.enable('mimetype')


In [15]:
# plot
source = df.groupby('your_undergraduate_core_field').count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs'})

alt.Chart(source).mark_bar().encode(
    alt.X('obs', title="Observations", scale=alt.Scale(type='log')),
    alt.Y('your_undergraduate_core_field', sort='-x', title="Field"),
    alt.Color('your_undergraduate_core_field', legend=None),
    tooltip = ['your_undergraduate_core_field', 'obs']
).properties(
    title = "Number of students per Undergraduate Core Field"
)

In [16]:
# plot 2
source = df.groupby(['your_undergraduate_core', 'your_undergraduate_core_field']).count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs'})
source
alt.Chart(source).mark_bar().encode(
    alt.X('obs', title="Observations", scale=alt.Scale(type='log')),
    alt.Y('your_undergraduate_core', sort='-x', title=""),
    alt.Color('your_undergraduate_core', sort='-x', legend=None),
    alt.Row('your_undergraduate_core_field', title=""),
    tooltip = ['your_undergraduate_core_field', 'your_undergraduate_core', 'obs']
# ).facet(
#     row='your_undergraduate_core_field',
).properties(
    title = "Number of students per Undergraduate Core and grouped Field"
).resolve_scale(y='independent', opacity='independent', color='independent')

In [17]:
print(df['your_undergraduate_core_field'].unique())

## Using SweetViz

In [ ]:
!pip install sweetviz

import sweetviz
df1 = df.copy()
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labels = df1['your_undergraduate_core'].unique()
i = 0
idx2class = {} 
class2idx = {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1
idx2class

df1['target_career_minor'] = df1['your_undergraduate_core'].replace(class2idx)
df1.head()

#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labelsM = df1['your_undergraduate_core_field'].unique()
i = 0
idx2classM = {} 
class2idxM = {}
for tp in labelsM:
    idx2classM[i] = tp
    class2idxM[tp] = i
    i += 1
idx2classM

df1['target_career_major'] = df1['your_undergraduate_core_field'].replace(class2idxM)
df1.head()
report1 = sweetviz.analyze(df1, "target_career_major")
report1.show_html()
report2 = sweetviz.analyze(df1, "target_career_minor")
report2.show_html()

# Building the Recommendation System

In [18]:
recsys_df = df.copy()
recsys_df.head()

In [19]:
# Final features
features_train = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
      'your_undergraduate_core',
      'your_undergraduate_core_field'
]


In [20]:
recsys = recsys_df[features_train]
recsys.head()

In [21]:
# Final features
features = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys[var].describe())
    print('-----')


In [22]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution='normal')

In [23]:
recsys['score_language_saber_11'] = qt.fit_transform(recsys[['score_language_saber_11']].values.reshape(-1,1))
recsys['score_mathematics_saber_11'] = qt.fit_transform(recsys[['score_mathematics_saber_11']].values.reshape(-1,1))
recsys['score_biology_saber_11'] = qt.fit_transform(recsys[['score_biology_saber_11']].values.reshape(-1,1))
recsys['score_chemistry_saber_11'] = qt.fit_transform(recsys[['score_chemistry_saber_11']].values.reshape(-1,1))
recsys['score_physics_saber_11'] = qt.fit_transform(recsys[['score_physics_saber_11']].values.reshape(-1,1))
recsys['score_social_science_saber_11'] = qt.fit_transform(recsys[['score_social_science_saber_11']].values.reshape(-1,1))
recsys['score_philosophy_saber_11'] = qt.fit_transform(recsys[['score_philosophy_saber_11']].values.reshape(-1,1))
recsys['score_english_saber_11'] = qt.fit_transform(recsys[['score_english_saber_11']].values.reshape(-1,1))

In [24]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys[var].describe())
    print('-----')

In [25]:
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labels = recsys['your_undergraduate_core'].unique()
i = 0
idx2class = {} 
class2idx = {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1
idx2class

**your_undergraduate_core or Minor Core** with 58 Classes

In [26]:
recsys['target_career_minor'] = recsys['your_undergraduate_core'].replace(class2idx)
recsys.head()

In [27]:
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labelsM = recsys['your_undergraduate_core_field'].unique()
i = 0
idx2classM = {} 
class2idxM = {}
for tp in labelsM:
    idx2classM[i] = tp
    class2idxM[tp] = i
    i += 1
idx2classM

**your_undergraduate_core_field or Major Core** with 10 Classes

In [28]:
recsys['target_career_major'] = recsys['your_undergraduate_core_field'].replace(class2idxM)
recsys.head()

In [29]:
# Final features
features_tr = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
    'target_career_minor',
     'target_career_major'
]

df1 = recsys[features_tr]
df1.head(2)

In [30]:
df1.target_career_major.unique()

In [31]:
def stratified_split(df, target, val_percent=0.2):
    '''
    Function to split a dataframe into train and validation sets, while preserving the ratio of the labels in the target variable
    Inputs:
    - df, the dataframe
    - target, the target variable
    - val_percent, the percentage of validation samples, default 0.2
    Outputs:
    - train_idxs, the indices of the training dataset
    - val_idxs, the indices of the validation dataset
    '''
    classes=list(df[target].unique())
    train_idxs, val_idxs = [], []
    for c in classes:
        idx=list(df[df[target]==c].index)
        np.random.shuffle(idx)
        val_size=int(len(idx)*val_percent)
        val_idxs+=idx[:val_size]
        train_idxs+=idx[val_size:]
    return train_idxs, val_idxs

In [32]:
train_idxs, val_idxs = stratified_split(df1, 'target_career_major', val_percent=0.20)

val_idxs, test_idxs = stratified_split(df1[df1.index.isin(val_idxs)], 'target_career_major', val_percent=0.2)

In [33]:
def test_stratified(df, col):
    '''
    Analyzes the ratio of different classes in a categorical variable within a dataframe
    Inputs:
    - dataframe
    - categorical column to be analyzed
    Returns: None
    '''
    classes=list(df[col].unique())
    
    for c in classes:
        print(f'Proportion of records with {c}: {len(df[df[col]==c])*1./len(df):0.2} ({len(df[df[col]==c])} / {len(df)})')

In [34]:
print('---------- STRATIFIED SAMPLING REPORT ----------')
print('-------- Label proportions in FULL data --------')
test_stratified(df1, 'target_career_major')
print('-------- Label proportions in TRAIN data --------')
test_stratified(df1[df1.index.isin(train_idxs)], 'target_career_major')
print('------ Label proportions in VALIDATION data -----')
test_stratified(df1[df1.index.isin(val_idxs)], 'target_career_major')
print('-------- Label proportions in TEST data ---------')
test_stratified(df1[df1.index.isin(test_idxs)], 'target_career_major')

In [35]:
train_df = df1[df1.index.isin(train_idxs)]
X_train = train_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_train = train_df[['target_career_major']].values
print('Retrieved Training Data')
val_df = df1[df1.index.isin(val_idxs)]
X_val = val_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_val = val_df[['target_career_major']].values
print('Retrieved Validation Data')
test_df = df1[df1.index.isin(test_idxs)]
X_test = test_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_test = test_df[['target_career_major']].values
print('Retrieved Test Data')

In [36]:
len(train_df),len(val_df),len(test_df),len(X_train),len(X_val),len(X_test),len(Y_train),len(Y_val),len(Y_test)

In [ ]:
#!pip install imblearn
#!pip install delayed

# Handling class imbalance using SMOTE based techniques
## A) SMOTE Technique

In [ ]:
#from imblearn.over_sampling import SMOTE
#from collections import Counter
# error : ImportError: cannot import name 'delayed' from 'sklearn.utils.fixes' (/opt/conda/lib/python3.7/site-packages/sklearn/utils/fixes.py)

#unique_b, counts_b = np.unique(Y_train, return_counts=True)

#counter = Counter(Y_train)
#print('Before',dict(zip(unique_b, counts_b)))

# oversampling the train dataset using SMOTE
#smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
#X_train_sm, y_train_sm = smt.fit_resample(X_train, Y_train)
#print()
#unique_a, counts_a = np.unique(y_train_sm, return_counts=True)
#counter = Counter(y_train_sm)
#print('After',dict(zip(unique_a, counts_a)))


In [ ]:
#X_val_sm, y_val_sm = smt.fit_resample(X_val, Y_val)
#X_test_sm, y_test_sm = smt.fit_resample(X_test, Y_test)

# Neural Network

In [45]:
# multi-class classification with Keras
import tensorflow as tf
#import visualkeras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout,Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [38]:
Y_train

In [39]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
encoded_Y_val = encoder.transform(Y_val)
encoded_Y_test = encoder.transform(Y_test)


In [40]:
encoded_Y_train

In [41]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = tf.keras.utils.to_categorical(encoded_Y_train)
dummy_y_val = tf.keras.utils.to_categorical(encoded_Y_val)
dummy_y_test = tf.keras.utils.to_categorical(encoded_Y_test)

In [42]:
dummy_y_train

In [ ]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=8, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(13, activation='softmax'))

# Metrics 

metrics = [ tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),
            'AUC',
            tf.keras.metrics.AUC(name='precision-recall_curve', curve='PR'), # precision-recall curve
            'top_k_categorical_accuracy',
            tf.keras.metrics.Precision(name="precision@5",top_k=5),
            tf.keras.metrics.Recall(name="recall@5",top_k=5)]
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics= metrics)

In [43]:
def get_model(act="relu"):
    model = Sequential()
    model.add(Dense(14, input_dim=8, activation=act))
    model.add(BatchNormalization())
    model.add(Flatten())

    model.add(Dense(100, activation=act))
    model.add(BatchNormalization())

    model.add(Dense(100, activation=act))
    model.add(BatchNormalization())

    model.add(Dense(100, activation=act))
    model.add(BatchNormalization())
  

    model.add(Dense(100, activation=act))
    model.add(BatchNormalization())

    model.add(Dense(10, activation='sigmoid'))
    model.summary()
    
    return model

In [46]:
# Metrics 
model = None
model = get_model()

metrics = [ tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),
            'AUC',
            tf.keras.metrics.AUC(name='precision-recall_curve', curve='PR'), # precision-recall curve
            'top_k_categorical_accuracy',
            tf.keras.metrics.Precision(name="precision@5",top_k=5),
            tf.keras.metrics.Recall(name="recall@5",top_k=5)]
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics= metrics)

In [47]:
#model = baseline_model()
model.summary()

In [ ]:
#visualkeras.layered_view(model)

In [48]:
%%time
history = model.fit(X_train,dummy_y_train, validation_data=(X_val,dummy_y_val),epochs= 20,batch_size=512,verbose=1)

In [49]:
print('----Model--------------------------------- \n')
print('------------\n')
print('Model Binary Accuracy = {}'.format(history.history['binary_accuracy'][np.argmin(history.history['loss'])]*100))
print('Model TOP 5 Accuracy = {}'.format(history.history['top_k_categorical_accuracy'][np.argmin(history.history['loss'])]*100))
print('Model AUC = {}'.format(history.history['auc'][np.argmin(history.history['loss'])]*100))
print('Model AUC-precision-recall_curve = {}'.format(history.history['precision-recall_curve'][np.argmin(history.history['loss'])]*100))
print('Model Precision = {}'.format(history.history['precision@5'][np.argmin(history.history['loss'])]*100))
print('Model Recall = {}'.format(history.history['recall@5'][np.argmin(history.history['loss'])]*100))
print('Model loss RMSE  = {}'.format(history.history['loss'][np.argmin(history.history['loss'])]))

In [50]:
# Evaluation on Train
model.evaluate(X_train,dummy_y_train)

In [51]:
# Evaluation on Vali
model.evaluate(X_val,dummy_y_val)

In [52]:
# Evaluation on Test
model.evaluate(X_test,dummy_y_test)

In [53]:
plt.figure(figsize=(20,16))
plt.subplot(411)
plt.title('binary_accuracy scores')
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.legend(['binary_accuracy', 'val_binary_accuracy'])

plt.subplot(412)
plt.title('AUC value')
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.legend(['auc', 'val_auc'])

plt.subplot(413)
plt.title('AUC Precision-Recall_Curve value')
plt.plot(history.history['precision-recall_curve'])
plt.plot(history.history['val_precision-recall_curve'])
plt.legend(['precision-recall_curve', 'val_precision-recall_curve'])


plt.subplot(414)
plt.title('top_k_categorical_accuracy value')
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.legend(['top_k_categorical_accuracy', 'val_top_k_categorical_accuracy'])

plt.subplot(445)
plt.title('Precision@5 value')
plt.plot(history.history['precision@5'])
plt.plot(history.history['val_precision@5'])
plt.legend(['precision@5', 'val_precision@5'])


plt.subplot(446)
plt.title('Recall@5 value')
plt.plot(history.history['recall@5'])
plt.plot(history.history['val_recall@5'])
plt.legend(['recall@5', 'val_recall@5'])


plt.subplot(448)
plt.title('RMSE Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.savefig('metrics_plots.jpg')
plt.show()

In [54]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_test, y_pred_bool))

In [55]:
y_pred

In [56]:
np.argmax(y_pred[0])


In [57]:
np.argmax(y_pred[1])

# Saving Model

In [61]:
model.save("model_career_RS_10classes.h5")

In [68]:
test = [90, 60, 50, 18, 89, 70, 66,49]
test = qt.fit_transform(np.array(test).reshape(1,-1))
y_v = model.predict([test])
np.argmax(y_v)

# SHAP & LIME

In [69]:
import lime
import lime.lime_tabular

In [72]:
recsys.head()

In [74]:
recsys.columns

class_names=['BUSINESS' 'SPORTS AND PHYSICAL TRAIN' 'ENGINEERING'
                                                                 'HUMANITIES AND SOCIAL SCIENCE' 'MATH AND PHYSICAL SCIENCES'
                                                                 'NUTRITION AND DIETETICS' 'HEALTH & MEDICINE' 'ARTS AND DESIGN'
                                                                 'BIOLOGICAL SCIENCE' 'PLASTIC ARTS, VISUAL ARTS'],

In [101]:
with open('RS_X_training.bin','wb') as f:
    np.save(f,X_train)
    f.close()

In [122]:
with open('RS_X_test.bin','wb') as f:
    np.save(f,X_test)
    f.close()

In [123]:
with open('RS_X_validation.bin','wb') as f:
    np.save(f,X_val)
    f.close()

# Lime Explanation

In [138]:
#Explain samples in test set
X_explain = test
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train,
                                                   feature_names=['score_language_saber_11', 'score_mathematics_saber_11',
                                                                   'score_biology_saber_11', 'score_chemistry_saber_11',
                                                                   'score_physics_saber_11', 'score_social_science_saber_11',
                                                                   'score_philosophy_saber_11', 'score_english_saber_11'],
                                                   discretize_continuous=True,
                                                   class_names=['BUSINESS','SPORTS AND PHYSICAL TRAIN','ENGINEERING',
                                                                 'HUMANITIES AND SOCIAL SCIENCE','MATH AND PHYSICAL SCIENCES',
                                                                 'NUTRITION AND DIETETICS','HEALTH & MEDICINE','ARTS AND DESIGN',
                                                                 'BIOLOGICAL SCIENCE','PLASTIC ARTS, VISUAL ARTS'],
                                                   
                                                   mode="classification",
                                                   verbose=True,
                                                   random_state=123)

#Explaining first subject in test set using all 8 features
exp = explainer.explain_instance(X_explain[0,:],
                                 model.predict, 
                                 num_features=8)
#Plot local explanation
exp.as_pyplot_figure()
#plt.tight_layout()
exp.show_in_notebook(show_table=True)

# SHAP Explanation

In [103]:
import shap

In [105]:
features=['score_language_saber_11', 'score_mathematics_saber_11','score_biology_saber_11', 'score_chemistry_saber_11',
                'score_physics_saber_11', 'score_social_science_saber_11','score_philosophy_saber_11', 'score_english_saber_11']

In [108]:
explainer = shap.Explainer(model,feature_names=features)

In [116]:
X_1 = {'score_language_saber_11':90, 
       'score_mathematics_saber_11':60, 
       'score_biology_saber_11':50, 
       'score_chemistry_saber_11':18, 
       'score_physics_saber_11':89, 
       'score_social_science_saber_11':70, 
       'score_philosophy_saber_11':66,
       'score_english_saber_11':49}


In [119]:
X_12 = np.array([X_1[feature] for feature in features]).reshape(-1, 1)
X_12

In [121]:
X_explain = X_12

In [ ]:
# explain the model's predictions on test set using SHAP values
# same syntax works for xgboost, LightGBM, CatBoost, and some scikit-learn models
explainer = shap.Explainer(model,feature_names=features)

# shap_values consists of a list of two matrices of dimension samplesize x #features
# The first matrix uses average nr of benign samples as base value
# The second matrix which is used below uses average nr of malignant samples as base value 
shap_values = explainer(X_explain)


# Interactive visualization of the explanation of the first subject
# in the test set (X_explain).
# It shows the relative contribution of features to get from the base value of malignant
# samples(average value)
# to the output value (1 in case of malignant sample)
# the numbers at the bottom show the actual values for this sample.
shap.initjs() #initialize javascript in cell
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X_explain.iloc[0,:])


In [ ]:
#Interactive visualization of all sample/feature Shapley values
#It is possible to show the relative contribution of individual features for all
# samples on the y-axis as well.
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], X_explain)

In [ ]:
#A summary plot with the shapley value (feature importance) 
shap.summary_plot(shap_values[1], X_explain)

In [ ]:
#Same as above, but with violin plots to better see the distribution of shapley values 
shap.summary_plot(shap_values[1], X_explain, plot_type="violin")